In [29]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import numpy as np
import pandas as pd
import random
import nltk
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
import re

In [30]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juanitobanano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juanitobanano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/juanitobanano/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
df_articulos = pd.read_json('./juve_datos.json',orient='record')
print(df_articulos.iloc[0].Titulo)

Presidente se reúne en Palacio Nacional con gobernador de Puebla


In [32]:
from unidecode import unidecode

df_articulos['Titulo_sin_acentos'] = df_articulos['Titulo'].apply(lambda x: unidecode(x).lower())
df_articulos['Titulo_sin_numeros'] = df_articulos['Titulo_sin_acentos'].apply(lambda x: re.sub(r'\d+', '', x))

In [33]:
df_articulos

,Titulo,Fecha,Titulo_sin_acentos,Titulo_sin_numeros
0,Presidente se reúne en Palacio Nacional con go...,17.04.2023,presidente se reune en palacio nacional con go...,presidente se reune en palacio nacional con go...
1,Obras complementarias en tramo 1 del Tren Maya...,17.04.2023,obras complementarias en tramo 1 del tren maya...,obras complementarias en tramo del tren maya ...
2,Fonatur y Semarnat rinden informe de avances e...,17.04.2023,fonatur y semarnat rinden informe de avances e...,fonatur y semarnat rinden informe de avances e...
3,"En ruta del Tren Maya, estudiantes de todos lo...",17.04.2023,"en ruta del tren maya, estudiantes de todos lo...","en ruta del tren maya, estudiantes de todos lo..."
4,Caminos pavimentados comunicarán todas las cab...,17.04.2023,caminos pavimentados comunicaran todas las cab...,caminos pavimentados comunicaran todas las cab...
...,...,...,...,...
3130,Presidente Andrés Manuel López Obrador encabez...,02.12.2018,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...
3131,Discurso del presidente Andrés Manuel López Ob...,02.12.2018,discurso del presidente andres manuel lopez ob...,discurso del presidente andres manuel lopez ob...
3132,Ceremonia de entrega del Bastón de Mando y Pri...,02.12.2018,ceremonia de entrega del baston de mando y pri...,ceremonia de entrega del baston de mando y pri...
3133,El Estado se ocupará de disminuir las desigual...,02.12.2018,el estado se ocupara de disminuir las desigual...,el estado se ocupara de disminuir las desigual...


In [34]:
import spacy
#python -m spacy download es_core_news_sm
nlp = spacy.load('es_core_news_sm')

df_articulos_lemm_norm = pd.DataFrame()

def process_title(title):
    doc = nlp(title)
    
    # Lemmatization
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
    
    
    return {'titulo_lemmatizado': ' '.join(lemmas)}

df_articulos['Titulo_lemmatizado'] = df_articulos['Titulo_sin_acentos'].apply(process_title).apply(pd.Series)

In [35]:
df_articulos

,Titulo,Fecha,Titulo_sin_acentos,Titulo_sin_numeros,Titulo_lemmatizado
0,Presidente se reúne en Palacio Nacional con go...,17.04.2023,presidente se reune en palacio nacional con go...,presidente se reune en palacio nacional con go...,presidente reunar palacio nacional gobernador ...
1,Obras complementarias en tramo 1 del Tren Maya...,17.04.2023,obras complementarias en tramo 1 del tren maya...,obras complementarias en tramo del tren maya ...,obra complementario tramo 1 tren maya superar ...
2,Fonatur y Semarnat rinden informe de avances e...,17.04.2023,fonatur y semarnat rinden informe de avances e...,fonatur y semarnat rinden informe de avances e...,fonatur semarnat rendir informe avance protecc...
3,"En ruta del Tren Maya, estudiantes de todos lo...",17.04.2023,"en ruta del tren maya, estudiantes de todos lo...","en ruta del tren maya, estudiantes de todos lo...",ruta tren maya estudiante nivel escolar accede...
4,Caminos pavimentados comunicarán todas las cab...,17.04.2023,caminos pavimentados comunicaran todas las cab...,caminos pavimentados comunicaran todas las cab...,camino pavimentado comunicarar cabecera munici...
...,...,...,...,...,...
3130,Presidente Andrés Manuel López Obrador encabez...,02.12.2018,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...
3131,Discurso del presidente Andrés Manuel López Ob...,02.12.2018,discurso del presidente andres manuel lopez ob...,discurso del presidente andres manuel lopez ob...,discurso presidente andres manuel lopez obrado...
3132,Ceremonia de entrega del Bastón de Mando y Pri...,02.12.2018,ceremonia de entrega del baston de mando y pri...,ceremonia de entrega del baston de mando y pri...,ceremonia entrega baston mando discurso nacion
3133,El Estado se ocupará de disminuir las desigual...,02.12.2018,el estado se ocupara de disminuir las desigual...,el estado se ocupara de disminuir las desigual...,ocupar disminuir desigualdad social seguirar d...


In [36]:
stopword_es = nltk.corpus.stopwords.words('spanish')
def remove_stopwords(text):
    texto = text.split()
    text = [word for word in texto if word not in stopword_es]
    text_r = ' '.join(text)
    return text_r
    
df_articulos['Titulo_ns'] = df_articulos['Titulo_sin_acentos'].apply(lambda x: remove_stopwords(x))

In [37]:
df_articulos

,Titulo,Fecha,Titulo_sin_acentos,Titulo_sin_numeros,Titulo_lemmatizado,Titulo_ns
0,Presidente se reúne en Palacio Nacional con go...,17.04.2023,presidente se reune en palacio nacional con go...,presidente se reune en palacio nacional con go...,presidente reunar palacio nacional gobernador ...,presidente reune palacio nacional gobernador p...
1,Obras complementarias en tramo 1 del Tren Maya...,17.04.2023,obras complementarias en tramo 1 del tren maya...,obras complementarias en tramo del tren maya ...,obra complementario tramo 1 tren maya superar ...,obras complementarias tramo 1 tren maya supera...
2,Fonatur y Semarnat rinden informe de avances e...,17.04.2023,fonatur y semarnat rinden informe de avances e...,fonatur y semarnat rinden informe de avances e...,fonatur semarnat rendir informe avance protecc...,fonatur semarnat rinden informe avances protec...
3,"En ruta del Tren Maya, estudiantes de todos lo...",17.04.2023,"en ruta del tren maya, estudiantes de todos lo...","en ruta del tren maya, estudiantes de todos lo...",ruta tren maya estudiante nivel escolar accede...,"ruta tren maya, estudiantes niveles escolares ..."
4,Caminos pavimentados comunicarán todas las cab...,17.04.2023,caminos pavimentados comunicaran todas las cab...,caminos pavimentados comunicaran todas las cab...,camino pavimentado comunicarar cabecera munici...,caminos pavimentados comunicaran todas cabecer...
...,...,...,...,...,...,...
3130,Presidente Andrés Manuel López Obrador encabez...,02.12.2018,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...,presidente andres manuel lopez obrador encabez...
3131,Discurso del presidente Andrés Manuel López Ob...,02.12.2018,discurso del presidente andres manuel lopez ob...,discurso del presidente andres manuel lopez ob...,discurso presidente andres manuel lopez obrado...,discurso presidente andres manuel lopez obrado...
3132,Ceremonia de entrega del Bastón de Mando y Pri...,02.12.2018,ceremonia de entrega del baston de mando y pri...,ceremonia de entrega del baston de mando y pri...,ceremonia entrega baston mando discurso nacion,ceremonia entrega baston mando primer discurso...
3133,El Estado se ocupará de disminuir las desigual...,02.12.2018,el estado se ocupara de disminuir las desigual...,el estado se ocupara de disminuir las desigual...,ocupar disminuir desigualdad social seguirar d...,"ocupara disminuir desigualdades sociales, segu..."


In [ ]:
def exportar_a_csv(text):
    texto = text.split()
    text = [word for word in texto if word not in stopword_es]
    text_r = ' '.join(text)
    return text_r

with open('palabras_por_documento.csv', 'w', encoding='utf-8') as palabras:
    palabras.write('id , palabra , fecha\n')
    c = 0

    for registro in df_articulos.iloc:
        fecha = registro[1]
        for palabra in registro[2].split():
            aux = '' + str(c) + ',' + palabra + ','+fecha+'\n'
            palabras.write(aux)
        c= c+1